In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 5.0 MB/s 
     |████████████████████████████████| 140 kB 73.9 MB/s 
     |████████████████████████████████| 384 kB 59.0 MB/s 
     |████████████████████████████████| 58 kB 5.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machin

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

/usr/local/lib/python3.8/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
# 뉴스페이지 날짜 12월 ["13","14","15","16","17","18","19","20"]
date_list = ["13","14","15","16","17","18","19","20"]
done_list = []
for i in date_list:
# 날짜 하나에 모든 페이지 불러오는 코드
    init_url = f"https://news.naver.com/main/list.naver?mode=LS2D&sid2=264&sid1=100&mid=shm&date=202212{i}&page=1" # 정치 - 대통령실 url
    resp = requests.get(init_url, headers={'User-agent': 'Mozilla/5.0'})

    page_list = [init_url]

    target_url = init_url

    while True:
        resp = requests.get(target_url, headers={'User-agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(resp.text)
        page_tags = soup.select('div.paging a')

        for page_tag in page_tags:
            url = page_tag.get('href') # ?mode=LS2D&sid2=264&sid1=100&mid=shm&date=20221222&page=10
            url = urljoin(init_url, url) # https://news.naver.com/main/list.naver?mode=LS2D&sid2=264&sid1=100&mid=shm&date=20221222&page=10
            
            if url not in page_list + done_list:
                page_list.append(url)

        # 뉴스리스트 긁어오기
        done_list.append(page_list.pop(0))
        if not page_list:
            break
        target_url = page_list[0]

        #다음페이지 접속하기



    # 한페이지 url 20개 - 위에 코드에 적용시켜야함

done_list

['https://news.naver.com/main/list.naver?mode=LS2D&sid2=264&sid1=100&mid=shm&date=20221213&page=1',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=264&sid1=100&mid=shm&date=20221213&page=2',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=264&sid1=100&mid=shm&date=20221213&page=3',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=264&sid1=100&mid=shm&date=20221213&page=4',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=264&sid1=100&mid=shm&date=20221213&page=5',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=264&sid1=100&mid=shm&date=20221213&page=6',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=264&sid1=100&mid=shm&date=20221213&page=7',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=264&sid1=100&mid=shm&date=20221213&page=8',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=264&sid1=100&mid=shm&date=20221213&page=9',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=264&sid1=100&mid=shm&date=20221213&page=10',

In [ ]:
# 네이버 뉴스 링크
news_links = []
for page_link in done_list:
    url  = page_link
    resp= requests.get(url,headers={'User-agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(resp.content, 'lxml')
    news_type1 = soup.select('ul.type06_headline li dt a')
    news_type2 = soup.select('ul.type06 li dl dt a')
 
    for i in range(len(news_type1)):
        news_links.append(news_type1[i]["href"])
    for j in range(len(news_type2)):
        news_links.append(news_type2[j]["href"])

# news_links_END= set(news_links) # 한페이지의 뉴스링크
news_links

['https://n.news.naver.com/mnews/article/001/0013640139?sid=100',
 'https://n.news.naver.com/mnews/article/001/0013640139?sid=100',
 'https://n.news.naver.com/mnews/article/001/0013640134?sid=100',
 'https://n.news.naver.com/mnews/article/001/0013640134?sid=100',
 'https://n.news.naver.com/mnews/article/001/0013640110?sid=100',
 'https://n.news.naver.com/mnews/article/001/0013640110?sid=100',
 'https://n.news.naver.com/mnews/article/079/0003717121?sid=100',
 'https://n.news.naver.com/mnews/article/079/0003717121?sid=100',
 'https://n.news.naver.com/mnews/article/422/0000574443?sid=100',
 'https://n.news.naver.com/mnews/article/422/0000574443?sid=100',
 'https://n.news.naver.com/mnews/article/422/0000574442?sid=100',
 'https://n.news.naver.com/mnews/article/422/0000574442?sid=100',
 'https://n.news.naver.com/mnews/article/020/0003467785?sid=100',
 'https://n.news.naver.com/mnews/article/020/0003467785?sid=100',
 'https://n.news.naver.com/mnews/article/014/0004941690?sid=100',
 'https://

In [ ]:
# 사진,타이틀 링크랑 중복 제거
news_links_END= list(set(news_links))

In [ ]:
# 기사정보 크롤링
articles= []
for i in news_links_END[:10]:
    url = f'{i}'

    resp= requests.get(url,headers={'User-agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(resp.content, 'lxml')

    percen= soup.select('span.u_cbox_chart_per')

    articles.append({'id':url[39:-8],
                     'article_url': url,
                    'title': soup.select('h2#title_area')[0].text,
                    'journalists': [i.text for i in soup.select('span.byline_s')],
                    'company':soup.select('a.media_end_head_top_logo img')[0]['alt'],
                    'time':soup.select('span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME')[0].text,
                    'content':soup.select('div#dic_area')[0].text,
                    'img': [i['data-src'] for i in soup.select('span.end_photo_org img')],
                    })    
pd.DataFrame(articles)

In [ ]:
# 댓글 크롤링

wd = webdriver.Chrome('chromedriver')

commentslist=[]
commentinfo=[]
commentcountinfo=[]
count=0
for url in news_links_END:
    comment_link= 'https://n.news.naver.com/mnews/article/comment/'+url[39:]
    print(comment_link)
    count+=1

    wd.implicitly_wait(3)
    wd.get(comment_link)
    time.sleep(1)

# 추가할 것 if 기사댓글을 막았을 때: continue  (반성)

    if count == 1:
        wd.find_element(By.CLASS_NAME,'u_cbox_cleanbot_setbutton').click()
        wd.find_element(By.ID,'cleanbot_dialog_checkbox_cbox_module').click()
        wd.find_element(By.CLASS_NAME,'u_cbox_layer_cleanbot2_extrabtn').click()
        time.sleep(1)
    

    while True:
        try:
            wd.find_element(By.CLASS_NAME,'u_cbox_btn_more').click()
            time.sleep(1)
            
        
        except:
            break

    soup = BeautifulSoup(wd.page_source, 'lxml')
    
    allcomments= soup.select('div.u_cbox_comment_box.u_cbox_type_profile')
    for acomment in allcomments:
        try:
        
            commentslist.append({
                            'article_id' : url[39:-8],
                            'id':acomment.select('span.u_cbox_nick')[0].text,
                            'time': acomment.select('span.u_cbox_date')[0].text,
                            'content':acomment.select('span.u_cbox_contents')[0].text,
                            'recomm':acomment.select('em.u_cbox_cnt_recomm')[0].text,
                            'unrecomm':acomment.select('em.u_cbox_cnt_unrecomm')[0].text})
        except:
            commentslist.append({
                                'article_id' : url[39:-8],
                                'id':acomment.select('span.u_cbox_nick')[0].text,
                                'time': acomment.select('span.u_cbox_date')[0].text,
                                'content': acomment.select('span.u_cbox_delete_contents')[0].text # '~이유로 댓글 삭제됨'
                                })
    
    
    comment_count= soup.select('ul.u_cbox_comment_count.u_cbox_comment_count3')
    countlist=comment_count[0].text.split(' ')
    commentcountinfo.append({'article_id' : url[39:-8],
                '현재 댓글': countlist[2].replace('작성자',''),
                '작성자 삭제': countlist[4].replace('규정',''),
                '규정 미준수': countlist[6]})
    
    percen= soup.select('span.u_cbox_chart_per')
    try:
        commentinfo.append({'article_id' : url[39:-8],
        'male':percen[0].text,
        'female':percen[1].text,
        '10':percen[2].text,
        '20':percen[3].text,
        '30':percen[4].text,
        '40':percen[5].text,
        '50':percen[6].text,
        '60':percen[7].text})
    except:
        commentinfo.append({'article_id' : url[39:-8]})
    print(count)

In [ ]:
# 댓글크롤링의 데이터 프레임
pd.DataFrame(commentslist)
# 아래 예시는 보기 편하게 하게 위한 샘플이다.

,article_id,id,time,content,recomm,unrecomm
0,008/0004828989,solo****,2022.12.15. 03:32,장모..무죄래.. 축하해. 잔액증명서 위조나 병원 보조금 불법 수령도 죄가 아니야...,0,1
1,008/0004828989,ddun****,2022.12.15. 00:39,꼴통이 지배하는 세상....참 엿같네,0,0
2,008/0004828989,navi****,2022.12.14. 22:46,술쳐머고 출근도 안하는 놈. 오늘 당장 디지게 해야 되는데,1,0
3,008/0004828989,navi****,2022.12.14. 22:40,모지리 색깔은 황. 저xx가 디져야 나라가 산다.,0,0
4,008/0004828989,6408****,2022.12.14. 11:05,존경하는 대통령님! 항상 응원하고 지지합니다. 좋은 정책으로 국민과 국익에 도움이 ...,0,3
...,...,...,...,...,...,...
667,018/0005387182,jjks****,2022.12.14. 09:45:35,주민 억압하는 원희룡이도 정치하도록 교체하라,0,2
668,018/0005387182,0320****,2022.12.14. 09:12:38,대통령을 바꿔야 나라가 살아 날텐데...,14,17
669,018/0005387182,leeh****,2022.12.14. 09:19:49,윤두창 경질 가능성은 없냐?,0,3
670,018/0005387182,kmg1****,2022.12.21. 08:34:21,작성자에 의해 삭제된 댓글입니다.,NaN,NaN


In [ ]:
# 댓글통계의 데이터프레임 - 댓글량이 어느정도 충족되지않으면 통계가 나오지않음 
pd.DataFrame(commentinfo)
# 아래 예시는 보기 편하게 하게 위한 샘플이다.

,article_id,male,female,10,20,30,40,50,60
0,003/0011601380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,008/0004828989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,001/0013645338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,029/0002772263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,421/0006521035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,421/0006522532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,469/0000712944,79%,21%,1%,10%,32%,25%,24%,9%
7,654/0000027846,76%,24%,0%,5%,19%,35%,26%,15%
8,018/0005387182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,277/0005192610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 댓글갯수의 데이터프레임
pd.DataFrame(commentcountinfo)
# 아래 예시는 보기 편하게 하게 위한 샘플이다.

,article_id,현재 댓글,작성자 삭제,규정 미준수
0,003/0011601380,0,0,0
1,008/0004828989,8,0,0
2,001/0013645338,0,0,0
3,029/0002772263,1,0,0
4,421/0006521035,35,6,0
5,421/0006522532,2,1,0
6,469/0000712944,187,20,1
7,654/0000027846,309,64,2
8,018/0005387182,31,7,0
9,277/0005192610,0,0,0
